In [1]:
# Keeping the plasmid data extractor function separately


In [ ]:
# Making function to create folders to store plasmid details for various hits
# separately, if the plasmid folder has already been created the function returns us the path to the folder

def create_new_folder(folder_name):
    # Check if the folder already exists
    if not os.path.exists(folder_name):
        # The path does not exist so we will be making a new folder
        # Create the folder
        try:
            os.mkdir(folder_name)
            return os.path.abspath(folder_name)
        except:
            # If for some reason we are unable to create a folder due to the name of the plasmid 
            # we will create a folder with a different name and store the hit result there
            if not os.path.exists(f"Miscellaneous_{folder_name}_end"):
                folder_name = f"Miscellaneous_{folder_name}_end"
                os.mkdir(folder_name)
                return os.path.abspath(folder_name)
            else:
#                 print(f"Folder weird_name already exists.")
                return os.path.abspath(f"Miscellaneous_{folder_name}_end")
    else:
#         print(f"Folder '{folder_name}' already exists.")
        return os.path.abspath(folder_name)



############ BETA VERSION OF THE ABOVE FUNCTION (WORK IN PROGRESS)################################

# Writing the function that will pull all the plasmid hits with updated criteria
# Set up a way to find the hits that contain the journalllll of publication and the author, webscrap to find citations
# add the hit to list only if citations > 10

# We also keep a track of which plasmids and accession number pass both of our filters

def plasmid_data_improved(plasmid_name_1, plasmid_email_id, filter_pass_list):

############## REMEMBER TO UNCOMMENT THIS PORTION WHEN RUNNING FINALLY ###################################
    # Making the folder for the plamsmid hits and storing it's path
    folder_name = plasmid_name_1
    if '.' in folder_name:
        folder_name = folder_name.split('.')
        temp_str = ''
        for sub_str in folder_name:
            temp_str += sub_str
            temp_str += '_'
#         folder_name = temp_str
    else:
        temp_str = folder_name
    path = create_new_folder(temp_str)
    print(f"Folder made for {folder_name} as {temp_str}")

##########################################################################################################

    # Setting up our query and accessing Entrez
    query = f"Plasmid+{plasmid_name_1}"
    email = plasmid_email_id
    Entrez.email = email

    # Searching GenBank now
    handle = Entrez.esearch(db = 'nucleotide', term = query, retmax = 100)
    record_id = Entrez.read(handle)['IdList']
    handle.close()
    # print(record_id)
    # Now storing the data we got in form of a dataframe
    # First making the dictionary to store the data

    plasmid_data = {"Accession No.": [], 'Organism': [], 'Topology': [], 'Plasmid': [], 'Nucleotide Sequence': [],
                  'Locus Tag': [], 'Gene Name': [], 'Gene Location': [], 'Gene Product': [], 'Gene Sequence': []}

    # Going through each record we got from the entrez esearch
    for records in record_id:
        # Extracting the genbank record for the id
        genbank_record = Entrez.efetch(db = 'nucleotide', id = records, rettype = 'gb', retmode = 'text')
        # Reading the record
        gb_record = SeqIO.read(genbank_record, 'genbank')
        # For each record that we retrieve we will perform some filtering to get the nucleotide sequence of our 
        # interest - seeing if the decription contains the plasmid name of interest and if it is complete or not
        
        # First order filtering (FILTER 1) - the record is non-empty and contains the keyword 'complete sequence'
        # along with the plasmid name
        if (len(gb_record.id) != 0 and len(gb_record.description) != 0) and len(gb_record.seq) != 0:
            
            # We will check if the sequence is of our plasmid of interest and complete or not, if yes then we proceed
            # Extracting Accession Number and description
            
            
            accession = [str(gb_record.annotations['accessions'][0])]
            description = gb_record.description

            ##### DEBUGGING PRINT STATEMENT ##############################
            print(f"Current hit: {plasmid_name_1} :- {description}")
            ################################################################


            # We are going to add the record only if it is complete and has the plasmid of interest
            if plasmid_name_1.lower() in description.lower() and ('complete sequence' in description.lower() or 'complete genome' in description.lower()):
                
                ##### DEBUGGING PRINT STATEMENT ##############################
#                 print(gb_record.annotations.get("references")[0].pubmed_id)
                ################################################################
                
                # SECOND ORDER FILTERING (FILTER 2.1) - the record must have a PubMed ID
                if not gb_record.annotations.get("references")[0].pubmed_id:
                    continue
                # Now we are sure that the record contains the plasmid name, is complete and has a pubmed id
                
                
                # SECOND ORDER FILTERING (FILTER 2.2) - the record must have more than 10 citations
                
                # Storing the pubmed id
                paper_pubmed_id = gb_record.annotations.get("references")[0].pubmed_id
                # Getting the paper title and the authors using previously defined function
                paper_info = get_paper_info(paper_pubmed_id)
                paper_title = paper_info['title']
                paper_authors = paper_info['authors']
                print(f"{paper_title} {paper_authors}")
                # Now getting the number of citations of the paper
                paper_citations = int(get_citations_count_from_google_scholar(paper_title, paper_authors))
                # Checking if the citations > 10
#                 print(f"{paper_title} {paper_authors}")
                print(paper_citations)
                
                if paper_citations < 10:
                    continue
                
#                 print(f"Current hit is published in {paper_title} by {', '.join(paper_authors)} and has {paper_citations} citations.")
                
                
                # Pausing the execution for a minute to not cross our daily query limit using API or being detected
                # by google scholar bot for querying too much
                pause_for_one_minute()
            
                
                # Now we continue, since we successfully know that the nucleotide sequence associated with the pubmed id
                # has > 10 citations as well, we will keep a record of the plasmid name and accession number that
                # surpass both the set filters. We will use it when using Abricate and Plasmid Finder
                
                if str(description.split("plasmid ")[-1].split(",")[0]) in filter_pass_list.keys():
                    filter_pass_list[str(description.split("plasmid ")[-1].split(",")[0])].append(accession[0])
                else:
                    filter_pass_list[str(description.split("plasmid ")[-1].split(",")[0])] = []
                
                
                # Now we know that the sequence has passed FILTER 1 and FILTER 2 we proceed making a dataframe. 
                organism = [str(gb_record.annotations["source"])]
                plasmid_name = [str(description.split("plasmid ")[-1].split(",")[0])]
                locus_tag = []
                topology = [str(gb_record.annotations['topology'])]
                gene_prdt = []
                gene_seq = []
                location = []
                gene_name = []

                ###########################################
                # Getting nucleotide sequence
                handle = Entrez.efetch(db="nucleotide", id=accession[0], rettype="fasta", retmode="text")
                fasta_sequence = handle.read()
                handle.close()
                fasta_seq_split = [x for x in fasta_sequence.split()]
                # To get the nucleotide sequence we get everything after 'complete sequence'
                # Finding complete sequence
                i = 1
                while fasta_seq_split[i-1] != 'sequence':        
                    i += 1
                # Now we are at the start of the nucleotide sequence we store all that follows
                nuc_seq = ''
                while i < len(fasta_seq_split):
                    nuc_seq += fasta_seq_split[i]
                    i += 1
                # print(nuc_seq)
                nucleotide_seq = [str(nuc_seq)]
                ###########################################################

                # Now we will access the record and store the bacteria name, accession number,
                # topology, plasmid, nucleotide sequence, locus tag for each gene with gene name present
                # and store their respective names, location, gene product thier sequence

                
                # First we access the features and get all the data of interest
                for features in gb_record.features:
                    if features.type == 'CDS':
                        # We first see if the qualifiers contains gene, if not then we skip that gene
                        if 'gene' in features.qualifiers.keys():
                            # In case translation, locus tag, gene product is missing then we simply leave that cell empty
                            # print(features.qualifiers.keys())
                            # Storing the gene name first
                            gene_name.append(str(features.qualifiers.get('gene')[0]))
                            # Now storing other data - locus tag, gene product, translation, sequence
                            if 'locus_tag' in features.qualifiers.keys():
                                locus_tag.append(str(features.qualifiers.get('locus_tag')[0]))
                            else:
                                locus_tag.append('NaN')
                            if 'product' in features.qualifiers.keys():
                                gene_prdt.append(str(features.qualifiers['product'][0]))
                            else:
                                gene_prdt.append('NaN')
                            if 'translation' in features.qualifiers.keys():
                                gene_seq.append(str(features.qualifiers['translation'][0]))
                            else:
                                gene_seq.append('NaN')
                        # Storing the location of the gene on the nucleotide sequenceeeeeeee
                        if features.type == 'gene':
                            location.append(str(features.location))

                # We will make the dataframe and store it only if all the columns are of the same length
                # print(len(locus_tag), len(locus_tag))
                if (len(locus_tag) == len(location) and len(locus_tag)!= 0):
                    plasmid_data["Accession No."] = accession*len(locus_tag)
                    plasmid_data["Organism"] = organism*len(locus_tag)
                    plasmid_data["Topology"] = topology*len(locus_tag)
                    plasmid_data["Plasmid"] = plasmid_name*len(locus_tag)
                    plasmid_data["Nucleotide Sequence"] = nucleotide_seq*len(locus_tag)
                    plasmid_data["Locus Tag"] = locus_tag
                    plasmid_data["Gene Length"] = [len(x) for x in gene_seq if x != 'NaN']
                    plasmid_data["Gene Name"] = gene_name
                    plasmid_data["Gene Sequence"] = gene_seq
                    plasmid_data["Gene Product"] = gene_prdt
                    plasmid_data['Gene Location'] = location
                
                
                # print(plasmid_data)
                # Making the dataframe
                dataframe = pd.DataFrame(plasmid_data)
                
                # Now we will use abricate to get the identified resistance to some (or all) genes 
                # as well present on the sequence
                
                # First saving the nucleotide sequence of the record as fasta file
                fasta_path = f"{accession[0]}_{plasmid_name[0]}.fa"
                with open(fasta_path, 'w') as file:
                    file.write(fasta_sequence)
                # Now using abricate on this saved file
                input_file_fas = fasta_path
                abricate_cmd = f'abricate {input_file_fas} --csv > {input_file_fas}_1.csv'
                result = subprocess.check_output(abricate_cmd, shell = True, universal_newlines = True)
                abr_df = pd.read_csv(f'{input_file_fas}_1.csv')
                abr_df.head()
                
                
                # Saving the dataframe
                dataframe.to_excel(f'{path}/{accession[0]}_{plasmid_name[0]}_data.xlsx', index = False)
                print(f"DataFrame made for {plasmid_name[0]} at /{accession[0]}")
                
#                 dataframe.head()
#             else:
#                 print("The hit does not pass the filter")